# **Importing Packages**

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
import itertools
import statsmodels.api as sm
import sklearn
import numpy as np
import seaborn as sns
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.utils import check_array
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from math import sqrt
from sklearn.model_selection import train_test_split
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from subprocess import check_call
from subprocess import call
from IPython.display import Image
!pip install graphviz


from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

# **Mounting Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Importing Data**

**Importing Training Data**

In [ ]:
print ('loading data')
train=pd.read_csv('/content/gdrive/MyDrive/Humana/original_dataset/2021_Competition_Training.csv')

loading data


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,8,9,11,13,16,20,26,28,29,31,33,51,55,58,62,64,66,68,75,85,102,124,127,131,132,135,160,174,180,187,192,202,209,210,211,215,220,230,234,240,243,247,251,255,261,285,293,297,300,305,306,309,323,334,344,345,352,353,355,359) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Importing DataType conversion list**

In [ ]:
data_type = pd.read_csv('/content/gdrive/MyDrive/Humana/name.csv')

# **Organizing Missing Value**

**Training Data**

In [ ]:
#read tick file
cate = data_type['categorical'].values.tolist()
cate = cate[0:102]
num = data_type["numerical"].values.tolist()

df_num = train[num]
df_cate = train[cate]

 
#replace all */nan
df_cate_new = df_cate.replace('*',np.nan)
df_cate_new = df_cate_new.astype(str)
df_cate_new.replace('nan',np.nan,inplace = True)

#replace * in numerical column with spac nan
df_num_new = df_num.replace('*',np.nan)
#transform numerical columns
df_num_new = df_num_new.astype(np.float)

train = pd.concat([df_cate_new,df_num_new],axis=1)  


In [ ]:
train.dtypes.value_counts()

float64    213
object     102
dtype: int64

In [ ]:
train.head(20)

,total_bh_copay_pmpm_cost_t_9-6-3m_b4,mcc_ano_pmpm_ct_t_9-6-3m_b4,rx_maint_pmpm_cost_t_12-9-6m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,rx_generic_pmpm_cost_t_6-3-0m_b4,bh_ncdm_ind,atlas_retirement_destination_2015_upda,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,auth_3mth_dc_snf,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,auth_3mth_acute_end,auth_3mth_psychic,atlas_hiamenity,auth_3mth_bh_acute,auth_3mth_acute_chf,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,mcc_chf_pmpm_ct_t_9-6-3m_b4,bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4,auth_3mth_dc_hospice,auth_3mth_acute_skn,rx_maint_pmpm_cost_t_6-3-0m_b4,cons_mobplus,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,atlas_hipov_1115,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_res,auth_3mth_acute_dig,auth_3mth_dc_acute_rehab,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,atlas_type_2015_mining_no,auth_3mth_post_acute_res,auth_3mth_acute_inf,rx_branded_pmpm_ct_t_6-3-0m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,atlas_low_employment_2015_update,auth_3mth_acute_cad,...,bh_physician_office_copay_pmpm_cost_6to9m_b4,rwjf_income_inequ_ratio,rej_total_physician_office_visit_ct_pmpm_0to3m_b4,credit_num_nonmtgcredit_60dpd,credit_bal_autofinance_new,rwjf_men_hlth_prov_ratio,rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4,cmsd2_sns_genitourinary_pmpm_ct,bh_ncal_pct,atlas_pct_snap16,ccsp_227_pct,atlas_ghveg_sqftpth12,rx_days_since_last_script_6to9m_b4,atlas_orchard_acrespth12,atlas_pct_laccess_multir15,cons_cgqs,ccsp_065_pmpm_ct,atlas_medhhinc,rwjf_mental_distress_pct,atlas_pct_laccess_nhpi15,credit_num_consumerfinance_new,rx_gpi2_49_pmpm_cost_0to3m_b4,cons_chva,atlas_avghhsize,rx_overall_net_paid_pmpm_cost_6to9m_b4,atlas_ownhomepct,atlas_orchard_farms12,atlas_pct_fmrkt_wic16,rx_gpi2_33_pmpm_ct_0to3m_b4,rwjf_social_associate_rate,atlas_freshveg_farms12,auth_3mth_post_acute_inf,days_since_last_clm_0to3m_b4,mcc_end_pct,cons_lwcm07,atlas_pct_fmrkt_otherfood16,atlas_pct_laccess_white15,auth_3mth_post_acute_mean_los,rx_gpi2_66_pmpm_ct,cons_ltmedicr
0,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,No Activity,Dec_1x-2x,0,0,Dec_1x-2x,Dec_1x-2x,0,No_Change,0,0,0,0,0,Dec_1x-2x,No Activity,No Activity,0,0,Dec_1x-2x,M,Inc_1x-2x,No Activity,0,No Activity,No Activity,0,0,0,No Activity,0,0,0,No Activity,No Activity,0,0,...,0.0,3.621404,0.0,0.621714,1389.427219,0.000400,0.000000,0.000000,0.0,12.083352,0.000000,147.994686,217.0,0.000000,1.260405,88.0,0.0,56861.0,0.103341,0.000000,0.073287,3.956667,158.0,2.12,38.320000,73.922397,0.0,0.0,0.333333,23.839718,5.0,0.0,120.0,0.000000,0.23359,0.000000,26.842903,0.0,0.000000,4.0
1,No Activity,No Activity,No Activity,New,No Activity,Inc_over_8x,0,0,Inc_1x-2x,Inc_1x-2x,0,New,0,0,0,0,0,Inc_1x-2x,No Activity,No_Change,0,0,Inc_1x-2x,S,No Activity,No Activity,0,No Activity,No Activity,0,0,0,No Activity,0,0,0,No Activity,No Activity,0,0,...,0.0,4.430692,0.0,0.549206,1795.154717,0.001456,0.333333,0.000000,0.0,8.677720,0.000000,80.357201,120.0,0.156425,0.310503,82.0,0.0,55828.0,0.106975,0.002699,0.228002,7.793334,254.0,2.58,0.000000,68.704457,21.0,0.0,0.333333,12.576445,69.0,0.0,120.0,0.000000,0.19198,50.000000,11.405128,0.0,0.000000,3.0
2,No Activity,No Activity,Dec_2x-4x,New,No Activity,Inc_1x-2x,0,0,Inc_1x-2x,Inc_1x-2x,0,No Activity,0,0,0,0,0,Inc_1x-2x,No Activity,No Activity,0,0,No_Change,S,Dec_2x-4x,No Activity,0,No Activity,Dec_4x-8x,0,0,0,Dec_2x-4x,0,0,0,No Activity,No Activity,0,0,...,0.0,3.682825,0.0,1.167164,1989.082598,0.000981,0.000000,0.000000,0.0,15.466672,1.958872,NaN,231.0,1.037510,0.773873,98.0,0.0,61903.0,0.113497,0.000000,0.502758,0.000000,118.0,2.57,1.186667,68.700726,10.0,0.0,0.000000,9.649039,14.0,0.0,36.0,0.000000,0.44111,0.000000,1.713138,0.0,0.083333,NaN
3,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,No Activity,New,0,0,Dec_2x-4x,No_Change,0,No Activity,0,0,0,0,0,Dec_1x-2x,No Activity,No Activity,0,0,Dec_2x

In [ ]:
### Function: Dropping Data
def datadrop(df,list,column):
  new_list = df.drop(list, axis=column)
  return new_list

## Function: Checking for Nulls or NA
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * mis_val / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

### Function: Identifying columns with only 1 unique value 
def unique_value_col(df):
        # Find the unique # of values in each column
        uni_val = []
        for col in trainingdata_1:
          if len(trainingdata_1[col].unique()) <= 1:
            uni_val.append(col) 
        # Return the dataframe with missing information
        return uni_val


### Function: drop columns with imbalance class problems
def drop_col_threshold(df,threshold):
    #length for each column
    length = df.shape[0]

    column_names = df.columns.values.tolist()
    column_remove = []

    #create a df for values of the highest frequency and percentage
    for name in column_names:
        for idx,val in enumerate(df[name].value_counts()[:1].index.tolist()):
            perc = df[name].value_counts()[idx] / length
            if perc >= threshold:
                column_remove.append(name)
    
    return(column_remove)

### Function: figure out variables with high categories
def cat_highlevel(df,threshold):

    column_names = df.columns.values.tolist()
    column_other_encode = []

    for col in df.select_dtypes(include='object'):
        if len(list(df[col].unique())) > threshold:
            column_other_encode.append(col)

    return(column_other_encode)
### Function: Confirming type
def conversion(df):
  data_type = pd.read_csv('gdrive/My Drive/Humana/name.csv')
  cate = data_type['categorical'].values.tolist()
  cate = cate[1:102]
  num = data_type["numerical"].values.tolist()


  df_num = df[num]
  df_cate = df[cate]


  df_cate = df_cate.astype(str)

  train = pd.concat([df_cate,df_num],axis=1)    
  return train


### **Removing problematic Columns**

In [ ]:
### Extracting 
trainingdata_missing_value = missing_values_table(train)
trainingdata_missing_value

Your selected dataframe has 315 columns.
There are 184 columns that have missing values.


,Missing Values,% of Total Values
mabh_seg,632924,64.9
cons_ltmedicr,312900,32.1
hedis_dia_hba1c_ge9,312228,32.0
atlas_ghveg_sqftpth12,260519,26.7
atlas_berry_acrespth12,252295,25.9
...,...,...
auth_3mth_dc_other,1,0.0
auth_3mth_acute_chf,1,0.0
auth_3mth_transplant,1,0.0
auth_3mth_dc_acute_rehab,1,0.0


In [ ]:
### drop column with % of Total value > threshold
threshold = 60
drop_table = trainingdata_missing_value[trainingdata_missing_value['% of Total Values']>= threshold].index
drop_table_a = list(drop_table)
print(drop_table_a)

['mabh_seg']


In [ ]:
train['mabh_seg'].unique()

array([nan, 'H7', 'C5', 'H6', 'C4', 'H1', 'H2', 'C2', 'H3', 'H4', 'H8',
       'C3', 'C6', 'C7', 'H5', 'C1'], dtype=object)

In [ ]:
### Dropping data from dataset
train_drop_missing = datadrop(train,drop_table_a,1)
train_drop_missing.head()

,total_bh_copay_pmpm_cost_t_9-6-3m_b4,mcc_ano_pmpm_ct_t_9-6-3m_b4,rx_maint_pmpm_cost_t_12-9-6m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,rx_generic_pmpm_cost_t_6-3-0m_b4,bh_ncdm_ind,atlas_retirement_destination_2015_upda,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,auth_3mth_dc_snf,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,auth_3mth_acute_end,auth_3mth_psychic,atlas_hiamenity,auth_3mth_bh_acute,auth_3mth_acute_chf,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,mcc_chf_pmpm_ct_t_9-6-3m_b4,bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4,auth_3mth_dc_hospice,auth_3mth_acute_skn,rx_maint_pmpm_cost_t_6-3-0m_b4,cons_mobplus,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,atlas_hipov_1115,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_res,auth_3mth_acute_dig,auth_3mth_dc_acute_rehab,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,atlas_type_2015_mining_no,auth_3mth_post_acute_res,auth_3mth_acute_inf,rx_branded_pmpm_ct_t_6-3-0m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,atlas_low_employment_2015_update,auth_3mth_acute_cad,...,bh_physician_office_copay_pmpm_cost_6to9m_b4,rwjf_income_inequ_ratio,rej_total_physician_office_visit_ct_pmpm_0to3m_b4,credit_num_nonmtgcredit_60dpd,credit_bal_autofinance_new,rwjf_men_hlth_prov_ratio,rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4,cmsd2_sns_genitourinary_pmpm_ct,bh_ncal_pct,atlas_pct_snap16,ccsp_227_pct,atlas_ghveg_sqftpth12,rx_days_since_last_script_6to9m_b4,atlas_orchard_acrespth12,atlas_pct_laccess_multir15,cons_cgqs,ccsp_065_pmpm_ct,atlas_medhhinc,rwjf_mental_distress_pct,atlas_pct_laccess_nhpi15,credit_num_consumerfinance_new,rx_gpi2_49_pmpm_cost_0to3m_b4,cons_chva,atlas_avghhsize,rx_overall_net_paid_pmpm_cost_6to9m_b4,atlas_ownhomepct,atlas_orchard_farms12,atlas_pct_fmrkt_wic16,rx_gpi2_33_pmpm_ct_0to3m_b4,rwjf_social_associate_rate,atlas_freshveg_farms12,auth_3mth_post_acute_inf,days_since_last_clm_0to3m_b4,mcc_end_pct,cons_lwcm07,atlas_pct_fmrkt_otherfood16,atlas_pct_laccess_white15,auth_3mth_post_acute_mean_los,rx_gpi2_66_pmpm_ct,cons_ltmedicr
0,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,No Activity,Dec_1x-2x,0,0.0,Dec_1x-2x,Dec_1x-2x,0,No_Change,0,0,0.0,0,0,Dec_1x-2x,No Activity,No Activity,0,0,Dec_1x-2x,M,Inc_1x-2x,No Activity,0.0,No Activity,No Activity,0,0,0,No Activity,0.0,0,0,No Activity,No Activity,0.0,0,...,0.0,3.621404,0.0,0.621714,1389.427219,0.000400,0.000000,0.0,0.0,12.083352,0.000000,147.994686,217.0,0.000000,1.260405,88.0,0.0,56861.0,0.103341,0.000000,0.073287,3.956667,158.0,2.12,38.320000,73.922397,0.0,0.0,0.333333,23.839718,5.0,0.0,120.0,0.0,0.23359,0.00,26.842903,0.0,0.000000,4.0
1,No Activity,No Activity,No Activity,New,No Activity,Inc_over_8x,0,0.0,Inc_1x-2x,Inc_1x-2x,0,New,0,0,0.0,0,0,Inc_1x-2x,No Activity,No_Change,0,0,Inc_1x-2x,S,No Activity,No Activity,0.0,No Activity,No Activity,0,0,0,No Activity,0.0,0,0,No Activity,No Activity,0.0,0,...,0.0,4.430692,0.0,0.549206,1795.154717,0.001456,0.333333,0.0,0.0,8.677720,0.000000,80.357201,120.0,0.156425,0.310503,82.0,0.0,55828.0,0.106975,0.002699,0.228002,7.793334,254.0,2.58,0.000000,68.704457,21.0,0.0,0.333333,12.576445,69.0,0.0,120.0,0.0,0.19198,50.00,11.405128,0.0,0.000000,3.0
2,No Activity,No Activity,Dec_2x-4x,New,No Activity,Inc_1x-2x,0,0.0,Inc_1x-2x,Inc_1x-2x,0,No Activity,0,0,0.0,0,0,Inc_1x-2x,No Activity,No Activity,0,0,No_Change,S,Dec_2x-4x,No Activity,0.0,No Activity,Dec_4x-8x,0,0,0,Dec_2x-4x,0.0,0,0,No Activity,No Activity,0.0,0,...,0.0,3.682825,0.0,1.167164,1989.082598,0.000981,0.000000,0.0,0.0,15.466672,1.958872,NaN,231.0,1.037510,0.773873,98.0,0.0,61903.0,0.113497,0.000000,0.502758,0.000000,118.0,2.57,1.186667,68.700726,10.0,0.0,0.000000,9.649039,14.0,0.0,36.0,0.0,0.44111,0.00,1.713138,0.0,0.083333,NaN
3,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,No Activity,New,0,0.0,Dec_2x-4x,No_Change,0,No Activity,0,0,0.0,0,0,Dec_1x-2x,No Activity,No Activity,0,0,Dec_2x-4x,M,In

In [ ]:
train_drop_missing.dtypes.value_counts()

float64    213
object     101
dtype: int64

### **Remove imbalance columns with values > 95%**




In [ ]:
drop_cols = drop_col_threshold(train_drop_missing.select_dtypes(include='object'),0.98)
train_drop_imbalance_features = datadrop(train_drop_missing,drop_cols,1)

In [ ]:
train_drop_imbalance_features.dtypes.value_counts()

float64    213
object      66
dtype: int64

In [ ]:
train_drop_st0 = train_drop_imbalance_features.copy()

In [ ]:
for col in train_drop_st0.select_dtypes("float64").columns:
    if train_drop_st0[col].describe()['std'] == 0:
        train_drop_st0 = train_drop_st0.drop(col,axis = 1)
        print(col)

auth_3mth_post_acute_dia
bh_ip_snf_net_paid_pmpm_cost_9to12m_b4
auth_3mth_acute_ckd
bh_ip_snf_net_paid_pmpm_cost_3to6m_b4
auth_3mth_post_acute_trm
auth_3mth_post_acute_rsk
auth_3mth_post_acute_ben
auth_3mth_acute_ccs_030
auth_3mth_acute_neo
auth_3mth_post_acute_cir
auth_3mth_transplant
auth_3mth_post_acute_inf


In [ ]:
cat = train_drop_st0.select_dtypes("object").columns
num = train_drop_st0.drop(cat,axis=1).columns


In [ ]:
new_type = pd.concat([pd.Series(cat),pd.Series(num)],axis=1)
new_type

,0,1
0,total_bh_copay_pmpm_cost_t_9-6-3m_b4,rx_gpi2_72_pmpm_cost_6to9m_b4
1,rx_maint_pmpm_cost_t_12-9-6m_b4,atlas_pct_laccess_child15
2,rx_nonbh_pmpm_cost_t_9-6-3m_b4,atlas_recfacpth14
3,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,atlas_pct_fmrkt_frveg16
4,rx_generic_pmpm_cost_t_6-3-0m_b4,atlas_pct_free_lunch14
...,...,...
196,NaN,atlas_pct_fmrkt_otherfood16
197,NaN,atlas_pct_laccess_white15
198,NaN,auth_3mth_post_acute_mean_los
199,NaN,rx_gpi2_66_pmpm_ct


In [ ]:
train_drop_st0.dtypes.value_counts()

float64    201
object      66
dtype: int64

In [ ]:
new_type.to_csv('new_type.csv')
!cp new_type.csv "/content/gdrive/MyDrive/Humana/new_type.csv"

In [ ]:
train_drop_st0.to_csv('train_post_drop.csv')
!cp train_post_drop.csv "/content/gdrive/MyDrive/Humana/train_post_drop.csv"